In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

import numpy as np
from matplotlib import pyplot as plt
import arrayfire as af

from dg_maxwell import utils
from dg_maxwell import lagrange
from dg_maxwell import params

af.set_backend(params.backend)

In [2]:
# Prototype implementation for Lp_Li
N   = 3

N_a = 3
foo = af.range(N * N_a, dtype = af.Dtype.u32)
foo = af.moddims(foo, d0 = N, d2 = 1, d1 = N_a)

N_b = 3
bar = af.range(N * N_b, dtype = af.Dtype.u32)
bar = af.moddims(bar, d0 = N, d2 = 1, d1 = N_b)

# print(foo)
# print(bar)

foo = af.reorder(foo, d0 = 0, d1 = 2, d2 = 1)
foo = af.transpose(af.tile(foo, d0 = 1, d1 = N))
foo = af.moddims(foo, d0 = N * N, d1 = 1, d2 = N)
foo = af.transpose(af.tile(foo, d0 = 1, d1 = N * N))
foo = af.transpose(af.moddims(af.transpose(foo), d0 = N * N * N * N, d1 = 1, d2 = N))
foo = af.reorder(foo, d0 = 2, d1 = 1, d2 = 0)
print(foo.shape)

bar = af.reorder(bar, d0 = 0, d1 = 2, d2 = 1)
bar = af.transpose(af.tile(bar, d0 = 1, d1 = N))
bar = af.moddims(bar, d0 = N * N, d1 = 1, d2 = N)
bar = af.tile(bar, d0 = 1, d1 = N * N)
bar = af.moddims(af.transpose(bar), d0 = N * N * N * N, d1 = 1, d2 = N)
bar = af.reorder(af.transpose(bar), d0 = 2, d1 = 1, d2 = 0)
# print(af.transpose(bar))

foo_bar = af.transpose(af.convolve1(foo, bar, conv_mode = af.CONV_MODE.EXPAND))
# print(foo_bar)

(3, 81)


In [3]:
# Prototype implementation for Lq_Lj
N   = 3

N_a = 3
foo = af.range(N * N_a, dtype = af.Dtype.u32)
foo = af.moddims(foo, d0 = N, d2 = 1, d1 = N_a)

N_b = 3
bar = af.range(N * N_b, dtype = af.Dtype.u32)
bar = af.moddims(bar, d0 = N, d2 = 1, d1 = N_b)

# print(foo)
# print(bar)

foo = af.reorder(foo, d0 = 0, d1 = 2, d2 = 1)
foo = af.tile(foo, d0 = 1, d1 = N)
foo = af.moddims(foo, d0 = N * N, d1 = 1, d2 = N)
foo = af.transpose(af.tile(foo, d0 = 1, d1 = N * N))
foo = af.moddims(af.transpose(foo), d0 = N * N * N * N, d1 = 1, d2 = N)
foo = af.reorder(foo, d0 = 2, d1 = 0, d2 = 1)
# print(foo.shape)

bar = af.reorder(bar, d0 = 0, d1 = 2, d2 = 1)
bar = af.tile(bar, d0 = 1, d1 = N)
bar = af.moddims(bar, d0 = N * N, d1 = 1, d2 = N)
bar = af.tile(bar, d0 = 1, d1 = N * N)
bar = af.moddims(af.transpose(bar), d0 = N * N * N * N, d1 = 1, d2 = N)
bar = af.reorder(af.transpose(bar), d0 = 2, d1 = 1, d2 = 0)
# print(bar)

foo_bar = af.transpose(af.convolve1(foo, bar, conv_mode = af.CONV_MODE.EXPAND))
print(foo_bar.shape)

(81, 5)


In [101]:
foo = af.reorder(foo, d0 = 0, d1 = 2, d2 = 1)
# foo = af.tile(foo, d0 = 1, d1 = N)
# foo = af.moddims()

In [102]:
print('original polynomials')
print(foo)
print(bar)

print('Poly1d square using convolve1')
# print(af.transpose(af.convolve1(af.transpose(foo), af.transpose(bar), conv_mode = af.CONV_MODE.EXPAND)))
# print(utils.poly1d_product(foo, bar))
print(af.transpose(af.convolve1(af.transpose(foo),
                                af.transpose(bar),
                                conv_mode = af.CONV_MODE.EXPAND)))


original polynomials
arrayfire.Array()
Type: unsigned int

[3 1 81 1]
         0 
         3 
         6 

         0 
         3 
         6 

         0 
         3 
         6 

         1 
         4 
         7 

         1 
         4 
         7 

         1 
         4 
         7 

         2 
         5 
         8 

         2 
         5 
         8 

         2 
         5 
         8 

         0 
         3 
         6 

         0 
         3 
         6 

         0 
         3 
         6 

         1 
         4 
         7 

         1 
         4 
         7 

         1 
         4 
         7 

         2 
         5 
         8 

         2 
         5 
         8 

         2 
         5 
         8 

         0 
         3 
         6 

         0 
         3 
         6 

         0 
         3 
         6 

         1 
         4 
         7 

         1 
         4 
         7 

         1 
         4 
         7 

         2 
         5 
         8 

     

In [19]:
for poly_foo, poly_bar in zip(foo, bar):
    foo_poly = np.poly1d(np.array(af.transpose(poly_foo)))
    bar_poly = np.poly1d(np.array(af.transpose(poly_bar)))
    print(foo_poly, end = '')
    print('\t<---------- original foo polynomial\n')
    print(bar_poly, end = '')
    print('\t<---------- original bar polynomial\n')

    print(foo_poly * bar_poly, end = '')
    print('\t<---------- foo_poly * bar_poly\n')
    print('\n*********************************************************\n')

 
3 x + 6	<---------- original foo polynomial

 
3 x + 6	<---------- original bar polynomial

   2
9 x + 36 x + 36	<---------- foo_poly * bar_poly


*********************************************************

   2
1 x + 4 x + 7	<---------- original foo polynomial

   2
1 x + 4 x + 7	<---------- original bar polynomial

   4     3      2
1 x + 8 x + 30 x + 56 x + 49	<---------- foo_poly * bar_poly


*********************************************************

   2
2 x + 5 x + 8	<---------- original foo polynomial

   2
2 x + 5 x + 8	<---------- original bar polynomial

   4      3      2
4 x + 20 x + 57 x + 80 x + 64	<---------- foo_poly * bar_poly


*********************************************************

